# Explore which companies should be excluded
As we've noticed some errors with our dataset, we need to ignore some companies.

This notebook BADLY needs refactoring. I think I can feel the hunger playing in...

## Load libraries

In [1]:
import pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import pyplot as plt
import time
import datetime
from dateutil.relativedelta import relativedelta
import requests
import locale
from pandas.io.json import json_normalize
import os
from os.path import exists
import sys
import io
from tqdm import tqdm

## Load custom libraries

In [2]:
import source.read_tickers_and_isins as URTI

### Define which data to be loaded

In [104]:
# Set flags for what will be handled
NAME = "Knudsen" # "Niedermayer"
#NAME = "Niedermayer"
prepare_and_download = True

# Constants:
## Different Parameters depending on the setting
if NAME == "Knudsen":
    no_https = False
    to_date_name = "DATE/TIME (DS End Date)"
    STOCK_EXCHANGE = "Nasdaq"
    n_input_files = 7
    _ticker = '%5EIXIC'  
elif NAME == "Niedermayer":
    no_https = True
    to_date_name = "DATE/TIME (DS End Date)"
    STOCK_EXCHANGE = "NYSE"
    n_input_files = 4
    _ticker = "%5Enya"
else:
    raise NotImplementedError

In [105]:
DATA_LOCATION = f'data/{NAME}/'
_insider_location = DATA_LOCATION + 'processed/insider/'

In [106]:
INPUT_FILE = f'input_data/{NAME}/{STOCK_EXCHANGE} Composite 16.3.2022 plus dead firms - {NAME}.xlsx'

In [107]:
data = URTI.read_tickers_and_isins(INPUT_FILE)

Reading tickers


### Start filtering

In [108]:
data['reason_to_exclude'] = 'None'

### Find non-unique tickers

In [116]:
dublicate_tickers = data[data.duplicated(subset=['TICKER SYMBOL'],keep=False)]['TICKER SYMBOL']
dublicate_tickers_mask = data['TICKER SYMBOL'].isin(dublicate_tickers)

In [117]:
data.loc[dublicate_tickers_mask, 'reason_to_exclude'] = 'Non-unique-ticker'

### Find NA tickers

In [118]:
mask = data['TICKER SYMBOL'] == 'NA'
data.loc[mask, 'reason_to_exclude'] = 'NA ticker'

### Show what we have

In [119]:
data

,Type,ISIN CODE,LOC OFF. CODE,NAME,DATASTREAM CODE,CUSIP,TICKER SYMBOL,BASE OR ST DATE,DATE/TIME (DS End Date),reason_to_exclude
0,2606T9,KYG870761080,UG87076108,10X CAPITAL VENTURE ACQUISITION II A,2606T9,G87076108,VCXA,2021-10-05,2022-03-16,None
1,95118Z,US88025U1097,U88025U109,10X GENOMICS A,95118Z,88025U109,TXG,2019-09-12,2022-03-16,None
2,9330V3,US68247Q1022,U68247Q102,111 ADR 1:2,9330V3,68247Q102,YI,2018-09-12,2022-03-16,None
3,99133U,US81807M2052,U81807M205,17 ED.TGP.ADR 1:10,99133U,81807M205,YQ,2020-12-04,2022-03-16,None
4,9101KM,US68236V1044,U68236V104,180 LIFE SCIENCES,9101KM,68236V104,ATNF,2017-06-27,2022-03-16,None
...,...,...,...,...,...,...,...,...,...,...
4073,30358V,US9898171015,U989817101,ZUMIEZ,30358V,989817101,ZUMZ,2005-05-06,2022-03-16,None
4074,50259R,US98880R1095,U98880R109,ZW DATA ACTION TECHNOLOGIES,50259R,98880R109,CNET,2007-10-15,2022-03-16,None
4075,2568RT,US98985X1000,U98985X100,ZYMERGEN,2568RT,98985X100,ZY,2021-04-22,2022-03-16,None
4076,98116P,US98986X1090,U98986X109,ZYNERBA PHARMACEUTICALS,98116P,98986X109,ZYNE,2015-08-05,2022-03-16,None


### Show amounts that are excluded

In [131]:
data.groupby(['reason_to_exclude']).count()

,Type,ISIN CODE,LOC OFF. CODE,NAME,DATASTREAM CODE,CUSIP,TICKER SYMBOL,BASE OR ST DATE,DATE/TIME (DS End Date)
reason_to_exclude,,,,,,,,,
NA ticker,158,158,158,158,158,158,158,158,158
Non-unique-ticker,93,93,93,93,93,93,93,93,93
None,3827,3827,3827,3827,3827,3827,3827,3827,3827


### Show which ones are excluded:

In [132]:
data[data['reason_to_exclude']!='None']

,Type,ISIN CODE,LOC OFF. CODE,NAME,DATASTREAM CODE,CUSIP,TICKER SYMBOL,BASE OR ST DATE,DATE/TIME (DS End Date),reason_to_exclude
18,2560C4,US35085K1097,U35085K109,4D PHARMA AMERICAN DEPOSITARY SHARES 1:8,2560C4,NA,NA,2021-03-22,2022-03-16,NA ticker
21,2654F6,US33830Q1094,U33830Q109,5E ADVANCED MATERIALS,2654F6,NA,NA,2022-03-15,2022-03-16,NA ticker
40,9070VE,US00372Y1055,U00372Y105,ABLYNX SPONSORED ADR 1:1 DEAD - DELIST.13/06/18,9070VE,NA,NA,2017-10-20,2018-06-12,NA ticker
58,2576J8,VGG0083E1025,UG0083E102,ACE GLOBAL BUSINESS ACQUISITION,2576J8,NA,NA,2021-05-26,2022-03-16,NA ticker
63,2619K4,US00444X1019,U00444X101,ACHARI VENTURES HOLDINGS I,2619K4,NA,NA,2021-11-17,2022-03-16,NA ticker
...,...,...,...,...,...,...,...,...,...,...
4005,9302NX,US98212N1072,U98212N107,WRAP TECHNOLOGIES,9302NX,NA,NA,2018-05-29,2022-03-16,NA ticker
4043,2640FQ,US98741L1017,U98741L101,YOSHITSU ADR 1:1,2640FQ,NA,NA,2022-01-18,2022-03-16,NA ticker
4054,91232M,US98920Y3045,U98920Y304,ZEALAND PHARMA ADR.1:1,91232M,NA,NA,2017-08-09,2022-03-16,NA ticker
4060,77170U,US98954M1018,U98954M101,ZILLOW GROUP CLASS A,77170U,98954M101,ZG,2011-07-20,2022-03-16,Non-unique-ticker


# Figure out a method to store results, so we filter in main notebook

In [133]:
isins_to_ignore = data[data['reason_to_exclude']!='None']['ISIN CODE'].to_list()
tickers_to_ignore = data[data['reason_to_exclude']!='None']['TICKER SYMBOL'].to_list()

In [142]:
print(f'ISINS: {len(isins_to_ignore)}')
print(f'tickers: {len(tickers_to_ignore)}')

ISINS: 251
tickers: 251


## Left to do
* Store these ISINs/tickers
* change analysis notebook to exclude them
* check how many insider tradings we have for the various tickers
* check how many we have time series for.
* Do some analysis/visualization of what we are taking out
  * Hypothesis: We are mainly taking out small companies (based on not a lot)

### Find which ISINS didn't have any Ticker associated:

In [143]:
DATA_LOCATION_RI = DATA_LOCATION + 'processed/RI_discard/'
_ri_location = DATA_LOCATION_RI

In [144]:
file_locs_ = os.listdir(_ri_location)
file_locs = [_ri_location + f for f in file_locs_]

In [145]:
drop_these = [_ri_location + isin + '.pickle' for isin in ISINs_to_ignore]

In [146]:
len(file_locs)

4077

In [147]:
keep_these = [x for x in file_locs if x not in drop_these]

In [148]:
len(keep_these)

4075

In [150]:
_insider_location

'data/Knudsen/processed/insider/'

In [27]:
#ticker = company.ticker
ticker = 'asdgjgff'

In [28]:
insider_data_df = pd.read_csv(_insider_location + ticker + '.csv', index_col=0, parse_dates=['FilingDate', 'TradeDate'])

FileNotFoundError: [Errno 2] No such file or directory: 'data/Niedermayer/processed/insider/asdgjgff.csv'

In [ ]:
insider_data_df